In [1]:
import pandas as pd
import numpy as np

# A. PMI data

In [2]:
# Import spreadsheet
df_pmi = pd.read_csv("Laporan Gudang Darurat PMI (s.d. 27 April 2021).csv")
df_pmi = df_pmi[df_pmi["Tanggal"].notna()]
df_pmi = df_pmi.rename(columns={'Tanggal': 'tanggal', 
                                'IN / OUT': 'inout', 
                                'Nomor GDN/GRN': 'gdngrn',
                                'Gudang': 'gudang',
                                'Dari/Kepada-Masking': 'darikepada', 
                               'Nama Barang': 'item',
                                'Merk/Jenis-Deleted': 'merk_jenis', 
                                'Satuan': 'satuan',
                                ' Masuk': 'in_jumlah', 
                                ' Keluar': 'out_jumlah', 
                                'Nilai': 'nilai_idr', 
                                ' Jumlah': 'nilaitotal_idr', 
                                'Berat': 'berat', 
                                'Total Berat': 'berattotal'
                               }) # Rename variables for consistency

print("\n### df_pmi head ###\n")
print(df_pmi.head())
      
print("\n### df_pmi dataframe info ###\n")
print(df_pmi.info())



### df_pmi head ###

     tanggal inout gdngrn gudang                  darikepada          item  \
0  01-Feb-20    IN  01.01     G2  Gudang Regional PMI_Serang  Masker Bedah   
1  01-Feb-20   OUT  01.01     G2                  Ketua Umum    Masker N95   
2  01-Feb-20   OUT  01.02     G2        PMI Kabupaten Natuna  Masker Bedah   
3  05-Feb-20   OUT  05.01     G2       Palang Merah Hongkong  Masker Bedah   
4  05-Feb-20   OUT  05.02     G2                   Lain-lain  Masker Bedah   

  merk_jenis satuan  in_jumlah out_jumlah    nilai_idr     nilaitotal_idr  \
0      Sensi    Pcs   350,000         NaN     Rp7,000    Rp2,450,000,000    
1         3M    Pcs        NaN       800    Rp108,240       Rp86,592,000    
2      Sensi    Pcs        NaN    20,000      Rp7,000      Rp140,000,000    
3      Sensi    Pcs        NaN    50,000      Rp7,000      Rp350,000,000    
4      Sensi    Pcs        NaN     4,000      Rp7,000       Rp28,000,000    

   berat  berattotal  
0  0.006      2100.0  


### 1. Sanity check and cleaning

#### Format Check
Based on early look at the dataset, several cleanups are necessary.
- Several datatypes require coercion to its proper class. Tanggal needed to be changed from object to datetime. in_jumlah, out_jumlah, nilai_idr, nilaitotal_idr, berat, in_jumlah needs to be converted to float.
- They also need to be cleaned from non-numeric characters (e.g. "Rp.", ",") to enable conversion

In [3]:
# Clean strings
df_pmi['in_jumlah'] = df_pmi['in_jumlah'].str.translate(str.maketrans({',': '', '-': '', ' ': ''}))
df_pmi['out_jumlah'] = df_pmi['out_jumlah'].str.translate(str.maketrans({',': '', '-': '', ' ': ''}))
df_pmi['nilai_idr'] = df_pmi['nilai_idr'].str.translate(str.maketrans({'R': '', 'p': '', ',': '', '-': '', ' ': ''}))
df_pmi['nilaitotal_idr'] = df_pmi['nilaitotal_idr'].str.translate(str.maketrans({'R': '', 'p': '', ',': '', '-': '', ' ': ''}))
df_pmi['berat'] = df_pmi['berat'].str.translate(str.maketrans({',': '', '-': '', ' ': ''}))
df_pmi[['in_jumlah', 'out_jumlah', 'nilai_idr', 'nilaitotal_idr', 'berat']].head()

,in_jumlah,out_jumlah,nilai_idr,nilaitotal_idr,berat
0,350000,NaN,7000,2450000000,0.006
1,NaN,800,108240,86592000,0.040
2,NaN,20000,7000,140000000,0.006
3,NaN,50000,7000,350000000,0.006
4,NaN,4000,7000,28000000,0.006


In [4]:
# Coerce to date
df_pmi['tanggal'] = pd.to_datetime(df_pmi['tanggal'])

# Coerce to float
df_pmi[['in_jumlah', 'out_jumlah', 'nilai_idr', 'nilaitotal_idr', 'berat']] = df_pmi[['in_jumlah', 'out_jumlah', 'nilai_idr', 'nilaitotal_idr', 'berat']].apply(pd.to_numeric)

# Check
print('\n### Check coerced columns ###\n')
print(df_pmi[['tanggal', 'in_jumlah', 'out_jumlah', 'nilai_idr', 'nilaitotal_idr', 'berat']].head())
print('\n### Check coerced columns datatypes###\n')
print(df_pmi[['tanggal', 'in_jumlah', 'out_jumlah', 'nilai_idr', 'nilaitotal_idr', 'berat']].info())


### Check coerced columns ###

     tanggal  in_jumlah  out_jumlah  nilai_idr  nilaitotal_idr  berat
0 2020-02-01   350000.0         NaN     7000.0    2.450000e+09  0.006
1 2020-02-01        NaN       800.0   108240.0    8.659200e+07  0.040
2 2020-02-01        NaN     20000.0     7000.0    1.400000e+08  0.006
3 2020-02-05        NaN     50000.0     7000.0    3.500000e+08  0.006
4 2020-02-05        NaN      4000.0     7000.0    2.800000e+07  0.006

### Check coerced columns datatypes###

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8189 entries, 0 to 8188
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tanggal         8189 non-null   datetime64[ns]
 1   in_jumlah       812 non-null    float64       
 2   out_jumlah      7373 non-null   float64       
 3   nilai_idr       7672 non-null   float64       
 4   nilaitotal_idr  7666 non-null   float64       
 5   berat           8094 non-null   

#### Null values check
A step to check whether there are any null values we should be concerned about.

In [5]:
df_pmi.isnull().sum()

tanggal              0
inout                0
gdngrn               1
gudang               0
darikepada           0
item                 0
merk_jenis        3845
satuan               2
in_jumlah         7377
out_jumlah         816
nilai_idr          517
nilaitotal_idr     523
berat               95
berattotal           0
dtype: int64

**Notes**
- merk_jenis: A lot of NAs due to many items not having further data beyond their names. This column should be dropped.
- in_jumlah, out_jumlah: are dependent on inout. If it's in, then out_jumlah has a NA (and vice versa). NAs should be replaced with 0, unless both in_jumlah and out_jumlah are NAs, in which case they are replaced and flagged as **"bermasalah"**.
- nilai_idr, nilaitotal_idr, berat: to be checked in the next step

In [6]:
# Drop merk_jenis
df_pmi = df_pmi.drop(columns = ['merk_jenis'])

In [7]:
# Replace NA in in_jumlah & out_jumlah
df_pmi[['in_jumlah', 'out_jumlah']] = df_pmi[['in_jumlah', 'out_jumlah']].fillna(0)

# Create net_jumlah to account for total in-coming and out-going stock
df_pmi['net_jumlah'] = df_pmi['in_jumlah'] - df_pmi['out_jumlah']

# Tag 
df_pmi['tag_jumlah'] = np.where(df_pmi['net_jumlah'] == 0, "Jumlah item bermasalah", "Clear")

# Check all variables
print('\n### Check clean variables ###\n')
print(df_pmi[['in_jumlah', 'out_jumlah', 'net_jumlah', 'tag_jumlah']])
print('\n### Check tagged variables ###\n')
print(df_pmi[['in_jumlah', 'out_jumlah', 'net_jumlah', 'tag_jumlah']].loc[df_pmi['tag_jumlah'] == 'Jumlah item bermasalah'])
print(df_pmi[['in_jumlah', 'out_jumlah', 'net_jumlah', 'tag_jumlah']].loc[df_pmi['tag_jumlah'] == 'Clear'])



### Check clean variables ###

      in_jumlah  out_jumlah  net_jumlah tag_jumlah
0      350000.0         0.0    350000.0      Clear
1           0.0       800.0      -800.0      Clear
2           0.0     20000.0    -20000.0      Clear
3           0.0     50000.0    -50000.0      Clear
4           0.0      4000.0     -4000.0      Clear
...         ...         ...         ...        ...
8184        0.0        10.0       -10.0      Clear
8185        0.0         9.0        -9.0      Clear
8186        0.0        46.0       -46.0      Clear
8187        0.0         2.0        -2.0      Clear
8188        0.0         2.0        -2.0      Clear

[8189 rows x 4 columns]

### Check tagged variables ###

      in_jumlah  out_jumlah  net_jumlah              tag_jumlah
565         0.0         0.0         0.0  Jumlah item bermasalah
1921        0.0         0.0         0.0  Jumlah item bermasalah
8167        0.0         0.0         0.0  Jumlah item bermasalah
8168        0.0         0.0         0.0  J

In [8]:
# Check null berat data
df_pmi[df_pmi.berat.isnull()].head(5)

,tanggal,inout,gdngrn,gudang,darikepada,item,satuan,in_jumlah,out_jumlah,nilai_idr,nilaitotal_idr,berat,berattotal,net_jumlah,tag_jumlah
254,2020-03-17,IN,17.08,G3,Gudang Regional PMI_Serang,Terpal,Lembar,445.0,0.0,NaN,NaN,NaN,0.0,445.0,Clear
1659,2020-03-28,IN,28.01,G3,Gudang Regional PMI_Serang,Matras Gulung,Pcs,100.0,0.0,NaN,NaN,NaN,0.0,100.0,Clear
2115,2020-04-02,OUT,02.10,G3,Tim Depo,Terpal,Lembar,0.0,4.0,NaN,NaN,NaN,0.0,-4.0,Clear
2545,2020-04-09,IN,09.02,G5,Swasta,Tisu Basah,Pack,144.0,0.0,16900.0,2433600.0,NaN,0.0,144.0,Clear
3321,2020-04-21,OUT,21.01,G3,PMI Kota Cirebon,Terpal,Lembar,0.0,1.0,NaN,NaN,NaN,0.0,-1.0,Clear


In [9]:
# Check where nilai_idr is not NA and nilaitotal is NA

df_pmi.loc[((df_pmi['nilai_idr'].isnull() == True) & (df_pmi['nilaitotal_idr'].isnull() == True))]

,tanggal,inout,gdngrn,gudang,darikepada,item,satuan,in_jumlah,out_jumlah,nilai_idr,nilaitotal_idr,berat,berattotal,net_jumlah,tag_jumlah
254,2020-03-17,IN,17.08,G3,Gudang Regional PMI_Serang,Terpal,Lembar,445.0,0.0,NaN,NaN,NaN,0.0,445.0,Clear
1659,2020-03-28,IN,28.01,G3,Gudang Regional PMI_Serang,Matras Gulung,Pcs,100.0,0.0,NaN,NaN,NaN,0.0,100.0,Clear
2115,2020-04-02,OUT,02.10,G3,Tim Depo,Terpal,Lembar,0.0,4.0,NaN,NaN,NaN,0.0,-4.0,Clear
3321,2020-04-21,OUT,21.01,G3,PMI Kota Cirebon,Terpal,Lembar,0.0,1.0,NaN,NaN,NaN,0.0,-1.0,Clear
3340,2020-04-21,IN,21.04,G7,Swasta,Obat - Bintang Toedjoe Panas Dalam,Pack,360.0,0.0,NaN,NaN,NaN,0.0,360.0,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8182,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Lay's Snack Potato 68 g,Pcs,0.0,15.0,NaN,NaN,NaN,0.0,-15.0,Clear
8183,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Chitato Snack 68 g,Pcs,0.0,5.0,NaN,NaN,NaN,0.0,-5.0,Clear
8184,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Kopi Kapal Api,Pcs,0.0,10.0,NaN,NaN,NaN,0.0,-10.0,Clear
8185,2021-04-26,OUT,-,G4,Tidak Layak digunakan (Reject),Mie Instan,Pcs,0.0,9.0,NaN,NaN,NaN,0.0,-9.0,Clear


**Note:**
- nilai_idr & berat: many are outright missing for no good reason. If missing, should be flagged as **"bermasalah"**.
- nilaitotal_idr: some are missing due to issues in in_jumlah and out_jumlah. However, some are missing when in_jumlah and out_jumlah are filled. As such, the calculated columns (nilaitotal_idr & berattotal) should all be refilled with raw calculation between absolute value of net_jumlah and their respective unit (nilai_idr and berat).  


In [10]:
# Replace NAs with 0 in nilai_idr and berat
df_pmi[['nilai_idr', 'berat']] = df_pmi[['nilai_idr', 'berat']].fillna(0)

# Add flagging condition for nilai_idr and berat
df_pmi['tag_nilai'] = np.where(df_pmi['nilai_idr'] == 0, "Nilai item bermasalah", "Clear")
df_pmi['tag_berat'] = np.where(df_pmi['berat'] == 0, "Berat item bermasalah", "Clear")

# Check tagging
print('\n### Check tagged variables nilai_idr ###\n')
print(df_pmi[['nilai_idr', 'tag_nilai']].loc[df_pmi['tag_nilai'] == 'Nilai item bermasalah'].head(5))
print(df_pmi[['nilai_idr', 'tag_nilai']].loc[df_pmi['tag_nilai'] == 'Clear'].head(5))
print('\n### Check tagged variables nilai_idr ###\n')
print(df_pmi[['berat', 'tag_berat']].loc[df_pmi['tag_berat'] == 'Berat item bermasalah'].head(5))
print(df_pmi[['berat', 'tag_berat']].loc[df_pmi['tag_berat'] == 'Clear'].head(5))


### Check tagged variables nilai_idr ###

      nilai_idr              tag_nilai
254         0.0  Nilai item bermasalah
1659        0.0  Nilai item bermasalah
2115        0.0  Nilai item bermasalah
3321        0.0  Nilai item bermasalah
3340        0.0  Nilai item bermasalah
   nilai_idr tag_nilai
0     7000.0     Clear
1   108240.0     Clear
2     7000.0     Clear
3     7000.0     Clear
4     7000.0     Clear

### Check tagged variables nilai_idr ###

      berat              tag_berat
254     0.0  Berat item bermasalah
1659    0.0  Berat item bermasalah
2115    0.0  Berat item bermasalah
2545    0.0  Berat item bermasalah
3321    0.0  Berat item bermasalah
   berat tag_berat
0  0.006     Clear
1  0.040     Clear
2  0.006     Clear
3  0.006     Clear
4  0.006     Clear


In [11]:
# Recalculate nilaitotal_idr and berattotal so that they wouyld represent net value
df_pmi['nilaitotal_idr'] = df_pmi['nilai_idr'] * df_pmi['net_jumlah']
df_pmi['berattotal'] = df_pmi['berat'] * df_pmi['net_jumlah']

# Add a new column for absolute value
df_pmi['abs_nilaitotal_idr'] = df_pmi['nilai_idr'] * abs(df_pmi['net_jumlah'])
df_pmi['abs_berattotal'] = df_pmi['berat'] * abs(df_pmi['net_jumlah'])

df_pmi[['nilai_idr', 'nilaitotal_idr', 'berat', 'berattotal', 'net_jumlah', 'abs_nilaitotal_idr', 'abs_berattotal']]

,nilai_idr,nilaitotal_idr,berat,berattotal,net_jumlah,abs_nilaitotal_idr,abs_berattotal
0,7000.0,2.450000e+09,0.006,2100.00,350000.0,2.450000e+09,2100.00
1,108240.0,-8.659200e+07,0.040,-32.00,-800.0,8.659200e+07,32.00
2,7000.0,-1.400000e+08,0.006,-120.00,-20000.0,1.400000e+08,120.00
3,7000.0,-3.500000e+08,0.006,-300.00,-50000.0,3.500000e+08,300.00
4,7000.0,-2.800000e+07,0.006,-24.00,-4000.0,2.800000e+07,24.00
...,...,...,...,...,...,...,...
8184,0.0,-0.000000e+00,0.000,-0.00,-10.0,0.000000e+00,0.00
8185,0.0,-0.000000e+00,0.000,-0.00,-9.0,0.000000e+00,0.00
8186,0.0,-0.000000e+00,0.000,-0.00,-46.0,0.000000e+00,0.00
8187,15500.0,-3.100000e+04,0.780,-1.56,-2.0,3.100000e+04,1.56


**Notes**
- Now that several issues have been tagged, we should add a column to indicate whether the row is cleared or has any issues whatsoever. This would help make an easier filter as to which data are added to the overall count. Only data that has no issue should be cleared for aggregation in the dashboard.

In [12]:
# Add overall issues tagging
df_pmi['tag_bermasalah'] = np.where((df_pmi['tag_nilai'] == 'Clear') & (df_pmi['tag_berat'] == 'Clear') & (df_pmi['tag_jumlah'] == 'Clear'),
                                    'Clear', 'Bermasalah')

# Check tags
df_pmi[['tag_nilai', 'tag_berat', 'tag_jumlah', 'tag_bermasalah']]

,tag_nilai,tag_berat,tag_jumlah,tag_bermasalah
0,Clear,Clear,Clear,Clear
1,Clear,Clear,Clear,Clear
2,Clear,Clear,Clear,Clear
3,Clear,Clear,Clear,Clear
4,Clear,Clear,Clear,Clear
...,...,...,...,...
8184,Nilai item bermasalah,Berat item bermasalah,Clear,Bermasalah
8185,Nilai item bermasalah,Berat item bermasalah,Clear,Bermasalah
8186,Nilai item bermasalah,Berat item bermasalah,Clear,Bermasalah
8187,Clear,Clear,Clear,Clear


### 2. Check columns

#### 2.1. Check shipment source/destination (darikepada). 
The aim of this section is to check the content of shipment destination and source. After the **darikepada** column is checked, it should form the basis of a variable to filter region as well as variable for PMI/NON-PMI. 

In [13]:
# Check unique values of shipment source/destination
df_darikepada= pd.DataFrame(df_pmi['darikepada'].value_counts())
print('\n### darikepada ###\n')
print(df_darikepada)

# Export csv for easier exploration
df_pmi['darikepada'].value_counts().to_csv('df_pmiloc.csv')

# Check 'PMI' in darikepada
print('\n### "darikepada" column which contains "PMI" before filtering out non-PMI ###\n')
print(df_pmi['darikepada'].loc[df_pmi['darikepada'].str.contains("PMI", case=False)].value_counts())


### darikepada ###

                             darikepada
Swasta                              779
MP PMI                              635
Tim Logistik                        468
Staff MP PMI                        390
Tim Depo                            273
...                                 ...
Tim Logistik Camp                     1
Satuan Pengawas Internal              1
Penyesuaian                           1
Produksi Paket Karyawan PMI           1
Non PMI - PBNU                        1

[390 rows x 1 columns]

### "darikepada" column which contains "PMI" before filtering out non-PMI ###

MP PMI                             635
Staff MP PMI                       390
PMI Kabupaten Bekasi               162
PMI Provinsi Jawa Timur             94
PMI Kota Bekasi                     92
                                  ... 
Gudang Regional PMI_Padang           1
Paket PHBS - PMI Kota Surakarta      1
Non PMI _ Smesco                     1
Non PMI - Panitia                    1
Non P

**Notes**

- We can see that there are generally two kinds of sources/destination, PMI and non-PMI. However, non-PMI sources/destination are sometimes written with their names as is (e.g."Swasta") or with "Non PMI" (e.g. "Non PMI AURI").The non-PMI sources/destinations should be easily identifiable by not having a PMI in their names, so the "Non PMI" portion of their names are to be omitted.   

- Gudang are also written with "PMI_", causing varying names for the same Gudang (e.g. Gudang Regional PMI_Serang and Gudang Regional Serang

In [14]:
# Remove 'Non PMI'
df_pmi['darikepada'] = df_pmi['darikepada'].str.replace('Non PMI', '')  # Replace strings
print('\n### "darikepada" after filtering out non-PMI ###\n')
print(df_pmi['darikepada'].loc[df_pmi['darikepada'].str.contains("PMI", case=False)].value_counts()) # Recheck PMI


# Make Gudang Regional naming uniform
df_pmi['darikepada'] = df_pmi['darikepada'].str.replace('PMI Gudang Regional', 'Gudang Regional')
df_pmi['darikepada'] = df_pmi['darikepada'].str.replace('PMI_', '')
df_pmi['darikepada'] = df_pmi['darikepada'].str.replace('Gudang Regional', 'Gudang Regional PMI')


# Gudang names are inconsistent (Gudang Serang and Banten are supposed to be the same, and Serang and Gresik are Gudang Sentral)
df_pmi['darikepada'] = df_pmi['darikepada'].str.replace('Gudang Regional PMI Banten', 'Gudang Regional PMI Serang')
df_pmi['darikepada'] = df_pmi['darikepada'].str.replace('Gudang Regional PMI Serang', 'Gudang Sentral PMI Serang')
df_pmi['darikepada'] = df_pmi['darikepada'].str.replace('Gudang Regional PMI Gresik', 'Gudang Sentral PMI Gresik')


print('\n### "darikepada" after reconfiguring Gudang names ###\n')
print(df_pmi['darikepada'].loc[df_pmi['darikepada'].str.contains("Gudang", case=False)].value_counts()) # Recheck Gudang Regional


### "darikepada" after filtering out non-PMI ###

MP PMI                        635
Staff MP PMI                  390
PMI Kabupaten Bekasi          162
PMI Provinsi Jawa Timur        94
PMI Kota Bekasi                92
                             ... 
PMI Kabupaten Bangka Barat      1
PMI Kabupaten Rote Ndao         1
PMI Kota Jayapura               1
PP Markas Pusat PMI             1
PMI Kabupaten Pati              1
Name: darikepada, Length: 168, dtype: int64

### "darikepada" after reconfiguring Gudang names ###

Gudang Sentral PMI Serang          47
Gudang Regional PMI Banjarmasin    11
Gudang Sentral PMI Gresik          11
Gudang Regional PMI Makassar       10
Gudang Regional PMI Semarang       10
Gudang Regional PMI Padang         10
Area Gudang                         2
Gudang                              2
Name: darikepada, dtype: int64


**Notes**
- Gudang Sentral and Gudang Regional data should then be marked with a separate column denoting their Gudang type. Then, Gudang Sentral and Regional related rows should be duplicated, so we can calculate their inventory status from the current dataset, and make them easily filterable. 
- This will be done by slicing a copy of the dataframe, changing their gudang column to their gudang names as well as changing the inout column to in, since all shipments must be in-coming. Afterwards, the edited dataframe is appended to the main data.

In [15]:
# Slice Gudang Sentral and Gudang Regional to a different dataset
df_pmireg = df_pmi.loc[df_pmi['darikepada'].str.contains("Gudang Regional", case=False)]
df_pmisen = df_pmi.loc[df_pmi['darikepada'].str.contains("Gudang Sentral", case=False)]
df_pmiregsen = pd.concat([df_pmireg, df_pmisen])


# Add gudang type to Gudang Regional and Gudang Sentral data
df_pmiregsen['tipe_gudang'] = np.where(df_pmiregsen['darikepada'].str.contains("Gudang Regional", case=False) == True, 
                                       'Gudang Regional', 'Gudang Sentral')
# Add gudang type to main data
df_pmi['tipe_gudang'] = 'Gudang Darurat'

# Reverse inout, in_jumlah, and out_jumlah in Gudang Regional and Gudang Sentral data and adjust calculations 
df_pmiregsen['inout'] = np.where(df_pmiregsen['inout'] == 'IN', 
                                       'OUT', 'IN')
df_pmiregsen = df_pmiregsen.rename(columns={'in_jumlah' : 'out_jumlah', 
                                            'out_jumlah': 'in_jumlah'})
df_pmiregsen[['nilaitotal_idr', 'berattotal', 'net_jumlah']] = df_pmiregsen[['nilaitotal_idr', 'berattotal', 'net_jumlah']] *-1 

# Change gudang from G1-G7 to gudang name in Gudang Regional and Gudang Sentral dataset
df_pmiregsen['gudang'] = df_pmiregsen['darikepada']


# Check Gudang Regional and Gudang Sentral dataset
df_pmiregsen

,tanggal,inout,gdngrn,gudang,darikepada,item,satuan,out_jumlah,in_jumlah,nilai_idr,...,berat,berattotal,net_jumlah,tag_jumlah,tag_nilai,tag_berat,abs_nilaitotal_idr,abs_berattotal,tag_bermasalah,tipe_gudang
194,2020-03-16,OUT,16.07,Gudang Regional PMI Banjarmasin,Gudang Regional PMI Banjarmasin,Masker N95,Pcs,1000.0,0.0,108240.0,...,0.04,-40.0,-1000.0,Clear,Clear,Clear,108240000.0,40.0,Clear,Gudang Regional
1588,2020-03-27,IN,27.01,Gudang Regional PMI Banjarmasin,Gudang Regional PMI Banjarmasin,Sprayer,Unit,0.0,2.0,650000.0,...,7.30,14.6,2.0,Clear,Clear,Clear,1300000.0,14.6,Clear,Gudang Regional
1596,2020-03-27,IN,27.03,Gudang Regional PMI Makassar,Gudang Regional PMI Makassar,Sprayer,Unit,0.0,2.0,650000.0,...,7.30,14.6,2.0,Clear,Clear,Clear,1300000.0,14.6,Clear,Gudang Regional
1597,2020-03-27,IN,27.03,Gudang Regional PMI Semarang,Gudang Regional PMI Semarang,Sprayer,Unit,0.0,1.0,650000.0,...,7.30,7.3,1.0,Clear,Clear,Clear,650000.0,7.3,Clear,Gudang Regional
2388,2020-04-07,OUT,07.08,Gudang Regional PMI Padang,Gudang Regional PMI Padang,Masker N95,Pcs,4480.0,0.0,108240.0,...,0.04,-179.2,-4480.0,Clear,Clear,Clear,484915200.0,179.2,Clear,Gudang Regional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7648,2021-02-22,OUT,22.01,Gudang Sentral PMI Serang,Gudang Sentral PMI Serang,Hygiene Kit,Paket,300.0,0.0,0.0,...,1.50,-450.0,-300.0,Clear,Nilai item bermasalah,Clear,0.0,450.0,Bermasalah,Gudang Sentral
8014,2021-03-09,OUT,09.01,Gudang Sentral PMI Serang,Gudang Sentral PMI Serang,Kemeja Putih,Pcs,180.0,0.0,0.0,...,0.10,-18.0,-180.0,Clear,Nilai item bermasalah,Clear,0.0,18.0,Bermasalah,Gudang Sentral
8015,2021-03-09,OUT,09.01,Gudang Sentral PMI Serang,Gudang Sentral PMI Serang,Rompi PMI,Pcs,100.0,0.0,150000.0,...,0.50,-50.0,-100.0,Clear,Clear,Clear,15000000.0,50.0,Clear,Gudang Sentral
8119,2021-04-07,OUT,07.01,Gudang Sentral PMI Serang,Gudang Sentral PMI Serang,Rompi PMI,Pcs,55.0,0.0,150000.0,...,0.50,-27.5,-55.0,Clear,Clear,Clear,8250000.0,27.5,Clear,Gudang Sentral


In [17]:
# Merge main dataset with Gudang Sentral and Gudang Regional dataset
df_pmi_merged = pd.concat([df_pmi, df_pmiregsen])

# Check merged dataset
df_pmi_merged.groupby('gudang').count()

,tanggal,inout,gdngrn,darikepada,item,satuan,in_jumlah,out_jumlah,nilai_idr,nilaitotal_idr,berat,berattotal,net_jumlah,tag_jumlah,tag_nilai,tag_berat,abs_nilaitotal_idr,abs_berattotal,tag_bermasalah,tipe_gudang
gudang,,,,,,,,,,,,,,,,,,,,
G1,1960,1960,1959,1960,1960,1960,1960,1960,1960,1960,1960,1960,1960,1960,1960,1960,1960,1960,1960,1960
G2,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509,1509
G3,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997
G4,559,559,559,559,559,559,559,559,559,559,559,559,559,559,559,559,559,559,559,559
G5,2101,2101,2101,2101,2101,2100,2101,2101,2101,2101,2101,2101,2101,2101,2101,2101,2101,2101,2101,2101
G6,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230
G7,833,833,833,833,833,832,833,833,833,833,833,833,833,833,833,833,833,833,833,833
Gudang Regional PMI Banjarmasin,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
Gudang Regional PMI Makassar,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10


#### 2.2. Check item names (Item). 
Here item names are checked. Since item names vary significantly, the easiest way to assign categories is to manually sort and tag item names with categories. As such, the aim of this section is to check item names and create a key table for item names and their categories.

In [18]:
# Add new dataframe for item column
df_item = pd.DataFrame(df_pmi['item'].value_counts())
df_item = df_item.reset_index()
df_item = df_item.rename(columns = {"index": "item", 
                                   "Item": "jumlah"})

# Export to spreadsheet for manual sorting
df_item.to_csv('df_item.csv')

# Import manually sorted key table
df_itemkey = pd.read_csv('df_item_sort.csv')
print("\n### Check count of each category ###\n")
print(df_itemkey.groupby('kategori').nunique('item'))
print("\n### Check dataset ###\n")
print(df_itemkey)


### Check count of each category ###

                           item  jumlah
kategori                               
Air                           1       1
Alat pelindung diri          15      13
Bahan Kimia                  27      20
Kebutuhan MCK                16      13
Kebutuhan Medis               3       3
Lainnya                      26       9
Makanan                      41      17
Minuman                      22      17
Obat dan Suplemen            29      16
Paket Bantuan                39      17
Peralatan Penunjang Medis     9       7

### Check dataset ###

                                 item  jumlah                   kategori
0                        Masker Bedah     737        Alat pelindung diri
1                              Hazmat     455        Alat pelindung diri
2                             Sprayer     436  Peralatan Penunjang Medis
3                            Kacamata     382        Alat pelindung diri
4                               Wipol     368       

In [19]:
# Join category data with main table
df_pmi_merged = pd.merge(df_pmi_merged, df_itemkey, on = ['item'])
df_pmi_merged = df_pmi_merged.drop(columns = ['jumlah'])
df_pmi_merged

,tanggal,inout,gdngrn,gudang,darikepada,item,satuan,in_jumlah,out_jumlah,nilai_idr,...,berattotal,net_jumlah,tag_jumlah,tag_nilai,tag_berat,abs_nilaitotal_idr,abs_berattotal,tag_bermasalah,tipe_gudang,kategori
0,2020-02-01,IN,01.01,G2,Gudang Sentral PMI Serang,Masker Bedah,Pcs,350000.0,0.0,7000.0,...,2100.0,350000.0,Clear,Clear,Clear,2.450000e+09,2100.0,Clear,Gudang Darurat,Alat pelindung diri
1,2020-02-01,OUT,01.02,G2,PMI Kabupaten Natuna,Masker Bedah,Pcs,0.0,20000.0,7000.0,...,-120.0,-20000.0,Clear,Clear,Clear,1.400000e+08,120.0,Clear,Gudang Darurat,Alat pelindung diri
2,2020-02-05,OUT,05.01,G2,Palang Merah Hongkong,Masker Bedah,Pcs,0.0,50000.0,7000.0,...,-300.0,-50000.0,Clear,Clear,Clear,3.500000e+08,300.0,Clear,Gudang Darurat,Alat pelindung diri
3,2020-02-05,OUT,05.02,G2,Lain-lain,Masker Bedah,Pcs,0.0,4000.0,7000.0,...,-24.0,-4000.0,Clear,Clear,Clear,2.800000e+07,24.0,Clear,Gudang Darurat,Alat pelindung diri
4,2020-02-05,OUT,05.03,G2,Staff MP PMI,Masker Bedah,Pcs,0.0,200.0,7000.0,...,-1.2,-200.0,Clear,Clear,Clear,1.400000e+06,1.2,Clear,Gudang Darurat,Alat pelindung diri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8283,2021-03-19,IN,19.01,G3,Non profit organization,Panduan CTPS 3,Lembar,100.0,0.0,0.0,...,25.0,100.0,Clear,Nilai item bermasalah,Clear,0.000000e+00,25.0,Bermasalah,Gudang Darurat,Lainnya
8284,2021-03-19,IN,19.01,G3,Non profit organization,Panduan CTPS 4,Lembar,100.0,0.0,0.0,...,25.0,100.0,Clear,Nilai item bermasalah,Clear,0.000000e+00,25.0,Bermasalah,Gudang Darurat,Lainnya
8285,2021-03-19,IN,19.01,G3,Non profit organization,Panduan CTPS 5,Lembar,100.0,0.0,0.0,...,25.0,100.0,Clear,Nilai item bermasalah,Clear,0.000000e+00,25.0,Bermasalah,Gudang Darurat,Lainnya
8286,2021-04-24,IN,24.01,G4,Gudang Sentral PMI Serang,Paket Lebaran,Paket,1400.0,0.0,0.0,...,0.0,1400.0,Clear,Nilai item bermasalah,Berat item bermasalah,0.000000e+00,0.0,Bermasalah,Gudang Darurat,Paket Bantuan


### 3. Exploration

#### 3.1. Gudang total weight/value

In [20]:
# Check total weight
pt_merged = df_pmi_merged.groupby(['gudang', 'inout'])
pt_merged[['nilaitotal_idr', 'berattotal']].mean()

nilaitotal_idr    berattotal
gudang                          inout                              
G1                              IN       8.772479e+07    287.845791
                                OUT     -9.970939e+06    -31.587605
G2                              IN       3.196729e+08    597.302048
                                OUT     -3.355955e+07    -60.979056
G3                              IN       2.335545e+08   2088.273377
                                OUT     -1.794915e+07   -159.692484
G4                              IN       6.212187e+08   3585.567750
                                OUT     -1.077483e+08   -628.701722
G5                              IN       1.260925e+08   3812.955851
                                OUT     -1.168924e+07   -361.399928
G6                              IN       4.132559e+08  10158.300741
                                OUT     -5.131623e+07  -1258.364089
G7                              IN       3.826560e+07   1308.801353
                                OUT     -1.712083e+06    -68.103784
Gudang Regional PMI Banjarmasin IN       1.492640e+06      4.020000
                                OUT     -1.082400e+08    -40.000000
Gudang Regional PMI Makassar    IN       2.855280e+06      6.580000
Gudang Regional PMI Padang      IN       2.649578e+06      4.977778
                                OUT     -4.849152e+08   -179.200000
Gudang Regional PMI Semarang    IN       1.768300e+06      3.930000
Gudang Sentral PMI Gresik       IN       3.195940e+06      7.220000
                                OUT      0.000000e+00      0.000000
Gudang Sentral PMI Serang       IN       3.877011e+06     53.977778
                                OUT     -1.154204e+08   -376.582759

#### 3.2. Category total weight/value

In [27]:
# Check total weight
pt_merged = df_pmi_merged.groupby(['kategori', 'inout'])
pt_merged[['nilaitotal_idr', 'berattotal']].mean()

nilaitotal_idr   berattotal
kategori                  inout                             
Air                       IN       4.849167e+07   448.833333
                          OUT     -1.361520e+06   -12.377451
Alat pelindung diri       IN       2.622435e+08   431.220577
                          OUT     -2.214414e+07   -36.695752
Bahan Kimia               IN       1.757197e+08  5335.705108
                          OUT     -1.863013e+07  -566.004168
Kebutuhan MCK             IN       1.940198e+08  5202.296078
                          OUT     -3.694057e+07  -997.604715
Kebutuhan Medis           IN       7.225000e+07    80.833333
                          OUT     -4.816667e+06    -5.388889
Lainnya                   IN       8.150000e+06   189.918333
                          OUT     -5.470588e+05   -59.350588
Makanan                   IN       1.387357e+07   156.626536
                          OUT     -2.482983e+06   -29.079872
Minuman                   IN       4.034734e+07  1820.067065
                          OUT     -1.614625e+06   -68.407558
Obat dan Suplemen         IN       4.695105e+06    74.925895
                          OUT     -6.907004e+05   -10.834833
Paket Bantuan             IN       2.997244e+08  1546.219944
                          OUT     -9.787056e+07  -514.479261
Peralatan Penunjang Medis IN       2.957350e+08  2443.535000
                          OUT     -3.680753e+07  -303.223094

#### 4. Engineer running total

In [20]:
# Slice necessary variables
df_pmi_merged_rt = df_pmi_merged[['tanggal', 'gudang', 'kategori', 'nilaitotal_idr', 'berattotal']]


df_pmi_merged_rt.head(20)

,tanggal,gudang,kategori,nilaitotal_idr,berattotal
0,2020-02-01,G2,Alat pelindung diri,2.450000e+09,2100.0
1,2020-02-01,G2,Alat pelindung diri,-1.400000e+08,-120.0
2,2020-02-05,G2,Alat pelindung diri,-3.500000e+08,-300.0
3,2020-02-05,G2,Alat pelindung diri,-2.800000e+07,-24.0
4,2020-02-05,G2,Alat pelindung diri,-1.400000e+06,-1.2
5,2020-02-11,G2,Alat pelindung diri,-7.000000e+05,-0.6
6,2020-02-12,G2,Alat pelindung diri,-7.000000e+05,-0.6
7,2020-02-14,G2,Alat pelindung diri,-7.000000e+05,-0.6
8,2020-02-17,G2,Alat pelindung diri,-1.400000e+07,-12.0
9,2020-02-17,G2,Alat pelindung diri,-1.750000e+06,-1.5


In [21]:
# Create datetime column for filling
key_gd = df_pmi_merged['gudang'].unique()
key_kategori = df_pmi_merged['kategori'].unique()
key_tanggal = pd.date_range(df_pmi_merged['tanggal'].min(), df_pmi_merged['tanggal'].max())
print(key_gd)
print(key_kategori)
print(key_tanggal)

['G2' 'G1' 'G5' 'Gudang Regional PMI Padang'
 'Gudang Regional PMI Makassar' 'Gudang Regional PMI Banjarmasin'
 'Gudang Regional PMI Semarang' 'Gudang Sentral PMI Serang'
 'Gudang Sentral PMI Gresik' 'G3' 'G4' 'G6' 'G7']
['Alat pelindung diri' 'Bahan Kimia' 'Peralatan Penunjang Medis'
 'Paket Bantuan' 'Kebutuhan MCK' 'Air' 'Minuman' 'Obat dan Suplemen'
 'Makanan' 'Kebutuhan Medis' 'Lainnya']
DatetimeIndex(['2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
               '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08',
               '2020-02-09', '2020-02-10',
               ...
               '2021-04-18', '2021-04-19', '2021-04-20', '2021-04-21',
               '2021-04-22', '2021-04-23', '2021-04-24', '2021-04-25',
               '2021-04-26', '2021-04-27'],
              dtype='datetime64[ns]', length=452, freq='D')


In [22]:
# Make base 
key_base = pd.MultiIndex.from_product([list(key_gd), list(key_kategori), list(key_tanggal)], 
                                     names = ['gudang', 'kategori', 'tanggal'])
pt_base = pd.DataFrame({"dummy": range(len(key_base))}, index = key_base)
pt_base



dummy
gudang kategori            tanggal          
G2     Alat pelindung diri 2020-02-01      0
                           2020-02-02      1
                           2020-02-03      2
                           2020-02-04      3
                           2020-02-05      4
...                                      ...
G7     Lainnya             2021-04-23  64631
                           2021-04-24  64632
                           2021-04-25  64633
                           2021-04-26  64634
                           2021-04-27  64635

[64636 rows x 1 columns]

In [23]:
# Add daily sum dataset
pt_daily = df_pmi_merged_rt.groupby(['gudang', 'kategori', 'tanggal']).sum()
pt_daily


nilaitotal_idr  \
gudang                    kategori                  tanggal                      
G1                        Alat pelindung diri       2020-03-02      86691000.0   
                                                    2020-03-03      43419000.0   
                                                    2020-03-04      14850000.0   
                                                    2020-03-05    -150706000.0   
                                                    2020-03-12     -10038000.0   
...                                                                        ...   
Gudang Sentral PMI Serang Paket Bantuan             2021-04-24             0.0   
                          Peralatan Penunjang Medis 2020-03-31       1300000.0   
                                                    2020-04-01       1300000.0   
                                                    2020-04-20    -100375000.0   
                                                    2020-04-28    -392466250.0   

                                                                berattotal  
gudang                    kategori                  tanggal                 
G1                        Alat pelindung diri       2020-03-02       44.28  
                                                    2020-03-03      351.06  
                                                    2020-03-04        3.00  
                                                    2020-03-05     -364.48  
                                                    2020-03-12      -29.64  
...                                                                    ...  
Gudang Sentral PMI Serang Paket Bantuan             2021-04-24        0.00  
                          Peralatan Penunjang Medis 2020-03-31       14.60  
                                                    2020-04-01       14.60  
                                                    2020-04-20    -1100.00  
                                                    2020-04-28    -4301.00  

[2010 rows x 2 columns]

In [29]:
# Merge key and daily sum dataset and fillna with 0
pt_daily_merged = pd.merge(pt_base, pt_daily, how = 'left', on = ['gudang', 'kategori', 'tanggal']).fillna(0)
pt_daily_merged

dummy  nilaitotal_idr  berattotal
gudang kategori            tanggal                                      
G2     Alat pelindung diri 2020-02-01      0    2.223408e+09      1948.0
                           2020-02-02      1    0.000000e+00         0.0
                           2020-02-03      2    0.000000e+00         0.0
                           2020-02-04      3    0.000000e+00         0.0
                           2020-02-05      4   -4.876400e+08      -365.2
...                                      ...             ...         ...
G7     Lainnya             2021-04-23  64631    0.000000e+00         0.0
                           2021-04-24  64632    0.000000e+00         0.0
                           2021-04-25  64633    0.000000e+00         0.0
                           2021-04-26  64634    0.000000e+00         0.0
                           2021-04-27  64635    0.000000e+00         0.0

[64636 rows x 3 columns]

In [37]:
# Calculate rolling sum
pt_rolling = pt_daily_merged.groupby(level=0).cumsum()
pt_rolling

dummy  nilaitotal_idr  berattotal
gudang kategori            tanggal                                          
G2     Alat pelindung diri 2020-02-01          0    2.223408e+09     1948.00
                           2020-02-02          1    2.223408e+09     1948.00
                           2020-02-03          3    2.223408e+09     1948.00
                           2020-02-04          6    2.223408e+09     1948.00
                           2020-02-05         10    1.735768e+09     1582.80
...                                          ...             ...         ...
G7     Lainnya             2021-04-23  308748780    6.126969e+08    13491.71
                           2021-04-24  308813412    6.126969e+08    13491.71
                           2021-04-25  308878045    6.126969e+08    13491.71
                           2021-04-26  308942679    6.126969e+08    13491.71
                           2021-04-27  309007314    6.126969e+08    13491.71

[64636 rows x 3 columns]

# B. Kawalcovid data

In [23]:
# Import excel sheet from KawalCovid19's public dataset
df_covid = pd.read_excel("COVID-19 di Indonesia @kawalcovid19.xlsx", sheet_name = "Timeline")
df_covid.head(5)

,Total Kasus,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo,?,Unnamed: 36
0,2020-03-18 00:00:00,0,1,17,0,0,3,158,0,24,...,1,0,0,0,0,0,0,0,0,NaN
1,2020-03-19 00:00:00,0,1,27,0,0,5,210,0,26,...,2,0,0,0,0,0,0,0,0,NaN
2,2020-03-20 00:00:00,0,4,37,0,0,4,215,0,41,...,1,0,0,0,0,0,0,0,13,NaN
3,2020-03-21 00:00:00,0,3,43,0,0,5,267,0,55,...,1,0,0,0,0,0,0,0,10,NaN
4,2020-03-22 00:00:00,0,3,47,0,0,5,307,0,59,...,1,0,1,0,2,0,0,0,6,NaN


#### 1. Total active cases dataframe 

In [25]:
# Slice only the active cases  and rename Total Kasus to date
df_covidactive = df_covid.iloc[1131:1691, 0:35]
df_covidactive = df_covidactive.dropna()
df_covidactive = df_covidactive.rename(columns={"Total Kasus":"tanggal"})

# Check dataset (wide)
print('\n### Wide Dataset ###\n')
print(df_covidactive.head())

# Melt into long dataframe
df_covidactive = df_covidactive.melt(id_vars = "tanggal")
df_covidactive['value'] = pd.to_numeric(df_covidactive['value'])
df_covidactive['tanggal'] = pd.to_datetime(df_covidactive['tanggal'])
df_covidactive = df_covidactive.rename(columns={'variable': 'provinsi', 
                                       'value': 'total_kasus'})



# Check dataset (long)
print('\n### Dataset Info ###\n')
print(df_covidactive.info())
print('\n### Long Dataset ###\n')
print(df_covidactive.head())


### Wide Dataset ###

                  tanggal Aceh Bali Banten Babel Bengkulu DIY Jakarta Jambi  \
1131  2020-03-21 00:00:00    0    2     40     0        0   4     227     0   
1132  2020-03-22 00:00:00    0    1     43     0        0   4     256     0   
1133  2020-03-23 00:00:00    0    4     52     0        0   4     301     1   
1134  2020-03-24 00:00:00    0    4     60     0        0   4     370     1   
1135  2020-03-25 00:00:00    0    7     62     0        0  14     409     1   

     Jabar  ... Sulteng Lampung Riau Malut Maluku Papbar Papua Sulbar NTT  \
1131    47  ...       0       1    1     0      0      0     0      0   0   
1132    45  ...       0       1    1     0      1      0     2      0   0   
1133    45  ...       0       1    1     1      1      0     2      0   0   
1134    45  ...       0       1    2     1      1      0     3      0   0   
1135    58  ...       0       1    1     1      1      0     3      0   0   

     Gorontalo  
1131         0  
1132 

#### 2. Daily case update

In [26]:
# Slice only the active cases  and rename Total Kasus to date
df_coviddaily = df_covid.iloc[564:1129, 0:35]
df_coviddaily = df_coviddaily.fillna(0)
df_coviddaily = df_coviddaily.rename(columns={"Total Kasus":"tanggal"})

# Check dataset (wide)
print('\n### Wide Dataset ###\n')
print(df_coviddaily)

# Melt into long dataframe
df_coviddaily = df_coviddaily.melt(id_vars = "tanggal")
df_coviddaily['value'] = pd.to_numeric(df_coviddaily['value'])
df_coviddaily['tanggal'] = pd.to_datetime(df_coviddaily['tanggal'])
df_coviddaily = df_coviddaily.rename(columns={'variable': 'provinsi', 
                                       'value': 'kasus_harian'})

# Check dataset (long)
print('\n### Dataset Info ###\n')
print(df_coviddaily.info())
print('\n### Long Dataset ###\n')
print(df_coviddaily.head())



### Wide Dataset ###

        tanggal  Aceh  Bali  Banten  Babel  Bengkulu  DIY  Jakarta  Jambi  \
564  2020-03-15     0     0       0      0         0    0       19      0   
565  2020-03-16     0     0       1      0         0    0       14      0   
566  2020-03-17     0     0       0      0         0    0        0      0   
567  2020-03-18     0     0       4      0         0    1       30      0   
568  2020-03-19     0     0      10      0         0    2       52      0   
...         ...   ...   ...     ...    ...       ...  ...      ...    ...   
1124 2021-09-26    44    85      47     51         6   61      168     23   
1125 2021-09-27    63    67      24     36         1   60      103     11   
1126 2021-09-28    42    92      44     61         7   71      104     31   
1127 2021-09-29    62    88      47     59         1   75      177     19   
1128 2021-09-30    40    86      59     62         2   67      149     18   

      Jabar  ...  Sulteng  Lampung  Riau  Malut  Mal

#### 3. Daily recovery update

In [27]:

# Slice only the active cases  and rename Total Kasus to date
df_coviddailyrec = df_covid.iloc[2253:2812, 0:35]
df_coviddailyrec = df_coviddailyrec.fillna(0)
df_coviddailyrec = df_coviddailyrec.rename(columns={"Total Kasus":"tanggal"})

# Check dataset (wide)
print('\n### Wide Dataset ###\n')
print(df_coviddailyrec)

# Melt into long dataframe
df_coviddailyrec = df_coviddailyrec.melt(id_vars = "tanggal")
df_coviddailyrec['value'] = pd.to_numeric(df_coviddailyrec['value'])
df_coviddailyrec['tanggal'] = pd.to_datetime(df_coviddailyrec['tanggal'])
df_coviddailyrec = df_coviddailyrec.rename(columns={'variable': 'provinsi', 
                                       'value': 'sembuh_harian'})

# Check dataset (long)
print('\n### Dataset Info ###\n')
print(df_coviddailyrec.info())
print('\n### Long Dataset ###\n')
print(df_coviddailyrec.head())


### Wide Dataset ###

        tanggal  Aceh  Bali  Banten  Babel  Bengkulu  DIY  Jakarta  Jambi  \
2253 2020-03-21     0     0       0      0         0    1        3      0   
2254 2020-03-22     0     0       0      0         0    0        5      0   
2255 2020-03-23     0     0       0      0         0    0        1      0   
2256 2020-03-24     0     0       0      0         0    0        0      0   
2257 2020-03-25     0     0       0      0         0    0        0      0   
...         ...   ...   ...     ...    ...       ...  ...      ...    ...   
2807 2021-09-26   144   143      40    131         0  115      126     56   
2808 2021-09-27    86   136      45    106         2  156      251     62   
2809 2021-09-28   155   265      39    103         3  132      174     42   
2810 2021-09-29    74   180      49     90         8  125      193     27   
2811 2021-09-30   101   229      87     74        10  190      194      9   

      Jabar  ...  Sulteng  Lampung  Riau  Malut  Mal

#### 3. Join total cases with daily case and recovery  update data

In [28]:
df_coviddailymerged = pd.merge(df_coviddailyrec, df_coviddaily)
df_coviddailymerged['net_harian'] = df_coviddailymerged['kasus_harian'] - df_coviddailymerged['sembuh_harian']
df_coviddailymerged

,tanggal,provinsi,sembuh_harian,kasus_harian,net_harian
0,2020-03-21,Aceh,0,0,0
1,2020-03-22,Aceh,0,0,0
2,2020-03-23,Aceh,0,0,0
3,2020-03-24,Aceh,0,0,0
4,2020-03-25,Aceh,0,0,0
...,...,...,...,...,...
19001,2021-09-26,Gorontalo,10,6,-4
19002,2021-09-27,Gorontalo,10,8,-2
19003,2021-09-28,Gorontalo,48,1,-47
19004,2021-09-29,Gorontalo,11,1,-10


In [31]:
df_covid_merged = pd.merge(df_covidactive, df_coviddailymerged)
df_covid_merged

,tanggal,provinsi,total_kasus,sembuh_harian,kasus_harian,net_harian
0,2020-03-21,Aceh,0,0,0,0
1,2020-03-22,Aceh,0,0,0,0
2,2020-03-23,Aceh,0,0,0,0
3,2020-03-24,Aceh,0,0,0,0
4,2020-03-25,Aceh,0,0,0,0
...,...,...,...,...,...,...
19001,2021-09-26,Gorontalo,185,10,6,-4
19002,2021-09-27,Gorontalo,183,10,8,-2
19003,2021-09-28,Gorontalo,136,48,1,-47
19004,2021-09-29,Gorontalo,126,11,1,-10


# C. Export datasets

In [30]:
# Export pmi data
df_pmi_merged.to_csv('df_pmi_merged.csv', index = False)

# Export daily pmi data for rolling sum
pt_daily_merged.to_csv('pt_daily_merged.csv')

# Export kawalcovid data
df_covid_merged.to_csv('df_covid_merged.csv', index = False)

TASKS
